In [1]:
import pandas as pd
import re
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import sem

from sklearn import datasets, ensemble
from sklearn.inspection import permutation_importance
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.neural_network import MLPRegressor
from time import time
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import QuantileTransformer
from sklearn.neural_network import MLPRegressor
from sklearn.datasets import make_regression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.linear_model import BayesianRidge, LinearRegression
from scipy.stats import spearmanr

## need biopython to run: https://anaconda.org/anaconda/biopython
from Bio.SeqUtils import MeltingTemp as mt
from Bio.Seq import Seq
## need OligoArrayAux to run: http://www.unafold.org/Dinamelt/software/OligoArrayAux.php
## install: https://libraries.io/pypi/oligo-melting
import oligo_melting as OligoMelt

In [ ]:
sns.set()

## Import data and graph

In [ ]:
df = pd.read_csv("Doench2014_V2.csv")
plt.hist(df["sgRNA Normalized"], alpha=0.5, density=True, label='Normalized sgRNA scores')
plt.hist(df["sgRNA Score"], alpha=0.5, density=True, label='Raw sgRNA scores')
plt.ylabel('Probability density')
plt.title("Dataset Histogram sgRNA scores")
plt.legend()
df.head()

### Models

In [ ]:
features = df.columns.values
y_feat = "sgRNA Normalized"
features = np.delete(features,0)
features = np.delete(features,0)
features = np.delete(features,0)
features = np.delete(features,0)
features = np.delete(features,0)
features = np.delete(features,0)
x_feat = features

In [ ]:
y = df.loc[:,y_feat].values
x = df[x_feat].values
X_train, X_test, y_train, y_test = train_test_split(
    x, y, test_size=0.1
)

## 10-fold cross validation

In [ ]:
def cross_validate_regression(model, df, x_feat_list, y_feat, n_splits=10):
    # computes the spearman correlation coefficient for each cross validated data set
    # takes in a model (estimator object), pandas dataframe, x features and the y-feature to predict, as well
    # as the number of splits (default is 10)
    #
    # returns a list of spearman correlation coefficients with length equal to the input n_splits
    
    # extract data into matrix
    x = df.loc[:, x_feat_list].values
    y_true = df.loc[:, y_feat].values
    
    kfold = KFold(n_splits=n_splits)
    
    # initialize an empty array same size as y_true
    y_pred = np.empty_like(y_true)
    
    # correlation list
    scc_list = np.array([])
    
    for train_idx, test_idx in kfold.split(x, y_true):
        # get training data
        x_train = x[train_idx, :]
        y_true_train = y_true[train_idx]

        # get testing data
        x_test = x[test_idx, :]
        y_true_test = y_true[test_idx]

        # train on training data
        model.fit(x_train, y_true_train)

        # estimate each penguin's species
        y_pred[test_idx] = model.predict(x_test)
        
        scc_list = np.append(scc_list, spearmanr(y_pred[test_idx], y_true[test_idx]).correlation)
    
    return scc_list

### Gradient-Boosted Regression Tree

In [ ]:
params = {
    "n_estimators": 1000,
    "max_depth": 4,
    "min_samples_split": 5,
    "learning_rate": 0.01,
}

reg = ensemble.GradientBoostingRegressor(**params)
reg.fit(X_train, y_train)

mse = mean_squared_error(y_test, reg.predict(X_test))
print("The mean squared error (MSE) on test set: {:.4f}".format(mse))

In [ ]:
# NOTE: this will take a while (~10 minutes) to run
gbrt_cvssc = cross_validate_regression(reg, df, x_feat, y_feat, 10)
gbrt_cvssc

##### Graphs of GBRT

In [ ]:
test_score = np.zeros((params["n_estimators"],), dtype=np.float64)
for i, y_pred in enumerate(reg.staged_predict(X_test)):
    test_score[i] = reg.loss_(y_test, y_pred)

fig = plt.figure(figsize=(6, 6))
plt.subplot(1, 1, 1)
plt.title("Deviance")
plt.plot(
    np.arange(params["n_estimators"]) + 1,
    reg.train_score_,
    "b-",
    label="Training Set Deviance",
)
plt.plot(
    np.arange(params["n_estimators"]) + 1, test_score, "r-", label="Test Set Deviance"
)
plt.legend(loc="upper right")
plt.xlabel("Boosting Iterations")
plt.ylabel("Deviance")
fig.tight_layout()
plt.show()

In [ ]:
feature_importance = reg.feature_importances_
sorted_idx = np.argsort(feature_importance)
pos = np.arange(sorted_idx.shape[0]) + 0.5
fig = plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.barh(pos, feature_importance[sorted_idx], align="center")
# plt.yticks(pos, x_feat)[sorted_idx]
plt.title("Feature Importance (MDI)")

result = permutation_importance(
    reg, X_test, y_test, n_repeats=10, random_state=42, n_jobs=2
)
sorted_idx = result.importances_mean.argsort()
plt.subplot(1, 2, 2)
plt.boxplot(
    result.importances[sorted_idx].T,
    vert=False,
    labels=x_feat[sorted_idx], # this might break it
)
plt.title("Permutation Importance (test set)")
fig.tight_layout()
plt.show()

## Linear Regression

In [ ]:
ols = LinearRegression()
ols.fit(x, y)

mse = mean_squared_error(y_test, ols.predict(X_test))
print("The mean squared error (MSE) on test set: {:.4f}".format(mse))

In [ ]:
lr_cvssc = cross_validate_regression(ols, df, x_feat, y_feat, 10)
lr_cvssc

##### Graphs for LR

In [ ]:
lw = 2
plt.figure(figsize=(6, 5))
plt.title("Weights of the model")
plt.plot(ols.coef_, color="lightblue", linewidth=lw, label="Linear Regression estimate")
plt.xlabel("Features")
plt.ylabel("Values of the weights")
plt.legend(loc="best", prop=dict(size=12))

### Bayesian Ridge Regression

In [ ]:
# Fit the Bayesian Ridge Regression and an OLS for comparison
clf = BayesianRidge(compute_score=True)
clf.fit(x, y)

mse = mean_squared_error(y_test, clf.predict(X_test))
print("The mean squared error (MSE) on test set: {:.4f}".format(mse))

In [ ]:
brr_cvssc = cross_validate_regression(clf, df, x_feat, y_feat, 10)
brr_cvssc

##### Graphs

In [ ]:
lw = 2
plt.figure(figsize=(6, 5))
plt.title("Weights of the model")
# plt.plot(ols.coef_, color="lightblue", linestyle="--", label="Linear Regression estimate")
plt.plot(clf.coef_, color="lightgreen", linewidth=lw, label="Bayesian Ridge estimate")
plt.xlabel("Features")
plt.ylabel("Values of the weights")
plt.legend(loc="best", prop=dict(size=12))

## Neural Network Regressor

In [ ]:
print("Training MLPRegressor...")
tic = time()
est = make_pipeline(
    QuantileTransformer(),
    MLPRegressor(
        hidden_layer_sizes=(100, 100),
        learning_rate_init=0.01,
        early_stopping=True,
        random_state=0,
    ),
)
est.fit(X_train, y_train)
print(f"done in {time() - tic:.3f}s")
print(f"Test R2 score: {est.score(X_test, y_test):.2f}")

In [ ]:
nn_cvssc = cross_validate_regression(est, df, x_feat, y_feat, 10)

## Model Performance

In [ ]:
models = ['gbrt', 'lr', 'brr', 'nn']
values = [gbrt_cvssc.mean(), lr_cvssc.mean(), brr_cvssc.mean(), nn_cvssc.mean()]
# error = [sem(gbrt_cvssc), sem(lr_cvssc), sem(brr_cvssc), sem(nn_cvssc)]
error = [gbrt_cvssc.std(), lr_cvssc.std(), brr_cvssc.std(), nn_cvssc.std()]

plt.bar(models, values, yerr=error, alpha=0.6)